In [72]:
import pandas as pd
import json

pd.set_option('display.max_rows', 500)


In [79]:
from bs4 import BeautifulSoup
import requests as r

In [73]:
import os
import sys
import inspect
parentdir = '../../../'
sys.path.insert(0, parentdir) 
src = '../../../src'
sys.path.insert(0, src) 

In [74]:
from config import Config
from util.selenium_dispatcher import SeleniumDispatcher
from db.mongo_adapter import MongoAdapter

In [75]:
## Getting all urls for moneycontrol
client = MongoAdapter().get_mongo_client()

In [76]:
collection = client[Config.MONGO_DB][Config.NEWS_COLLECTION]

In [77]:
cur = collection.aggregate([
    {
        '$addFields':
        {
            "urlMatch":
            {
                '$regexMatch':
                        {
                            'input': "$url",
                            'regex': "moneycontrol"
                        }
            }
        }
    }
    ,
    {
        '$match':
        {
            'urlMatch': True
        }
    }
    ,
    {
        '$project':
        {
            'url': 1
        }
    }
])


In [65]:
objs = {}
for dat in cur:
    # print(dat)
    id = str(dat['_id'])
    url = dat['url']
    # print(id, url)
    objs[id] = url 

In [112]:
urls = list(objs.values())
ids = list(objs.keys())

In [117]:
dat = {}

for id, url in zip(ids, urls):

    res = r.get(url)
    html_str = res.content
    bs = BeautifulSoup(html_str, 'html.parser')
    title = bs.find_all(class_="article_title artTitle")[0].text
    subtitle = bs.find_all(class_="article_desc")[0].text

    content_div = bs.find_all('div', class_="content_wrapper arti-flow")
    content = ''
    for p in  content_div[0].find_all('p'):
        content = content + p.text + '\n'

    dat[id] = {
        'title': title,
        'subtitle': subtitle,
        'content': content
    }

In [118]:
dat

{'0068887183e6334febe4cece': {'title': 'XLRI completes final placement in two days, average salary up by Rs 2 lakh at Rs 24.3 lakh pa',
  'subtitle': 'The final recruitment process saw participation from 108 recruiters with 362 domestic and international offers.',
  'content': 'Xavier School of Management (XLRI) has been able to secure 100 percent placement for the outgoing batch in just two days. The average salary rose by Rs 1.95 lakh year-on-year to Rs 24.3 lakh per annum.\nThe final recruitment process saw participation from 108 recruiters with 362 domestic and international offers.\n"This year\'s placement season has once again witnessed remarkable success in spite of the longish economic slowdown. Interest in XLRI\'s students has been tremendous, with top companies coming for recruitment," said P Christie, Director, XLRI-Xavier School of Management.\nThe institute said that 43 percent of the batch got pre-placement offers. Here, the median salary offered to the batch saw an incre